In [1]:
import tensorflow

In [2]:
#곱셈 계층 MulLayer
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    def forward(self,x,y):
        self.x = x
        self.y = y
        out = x * y
    
        return out
    def backward(self,dout): # 상류에서 넘어온 미분(dout)에 순전파 때의 값을 서로 바꿔 곱한 후 하류로 흘림
        
        dx = dout * self.y # x와 y를 바꾼다.
        dy = dout * self.x
        
        return dx, dy
    
    
apple = 100
apple_num = 2
tax = 1.1

#계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

#순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price,tax)

print(price)

#역전파
dprice = 1 # 출력층의 미분값은 1
dapple_price, tax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num)

220.00000000000003
2.2 110.00000000000001


In [3]:
#덧셈계층 AddLayer
class AddLayer:
    def __init__(self):
        pass
    def forward(self,x,y):
        out = x+y
        return out
    def backward(self,dout):
        dx = dout *1#덧셈노드는 그대로이기 때문에
        dy = dout *1
        return dx,dy
    

apple = 100
apple_num =2
orange = 150
orange_num = 3
tax = 1.1

#계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

#순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange,orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

#역전파 
dprice = 1 #맨 끝에서 출발할땐 미분값이 1이기 때문에 
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

    

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


In [4]:
#활성화 함수 계층 구현하기 
# ReLU 계층!!!
#신경망을 구성하는 층인 계층은 각각을 클래스 하나로 구현
import tensorflow as tf
import numpy as np
class Relu:
    def __init__(self):
        self.mask = None###mask는 0보다 크면 False로 작으면 True로 형태로 배열 저장
    def foward(self, x):
        self.mask = (x<=0)
        out = x.copy()
        out[self.mask] = 0
    def backward(self,dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx
    
x = np.array([[1,-0.5],[-2,3]])
print(x)

mask = (x<=0)
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


In [5]:
#Sigmoid계층
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self,x):
        out = 1 /(1+np.exp(-x))
        self.out = out
        
        return out
    def backward(self,dout):
        dx = dout*(1-self.out)*self.out
        return dx
    ## 순전파의 출력을 out에다가 보관했다가 다시 역전파 계신시 그 값을 이용
    ###즉 dout * y(1-y)!!
    

In [6]:
#배치용 Affine계층
x_dot_W = np.array([[0,0,0],[10,10,10]])
B = np.array([1,2,3])

print(x_dot_W)

print(x_dot_W + B)
## 이렇게 되면 안됨 첫번째 열의 합이 더해져야함
dy = np.array([[1,2,3],[4,5,6]])
print(dy)

db = np.sum(dy,axis=0)
print(db)
##이렇게 해야 됨 

[[ 0  0  0]
 [10 10 10]]
[[ 1  2  3]
 [11 12 13]]
[[1 2 3]
 [4 5 6]]
[5 7 9]


In [7]:
##Affine 계층 클래스 


class Affine:
    def __init__(self,W,b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self,x):
        self.x = x
        out = np.dot(x,self.W)+self.B
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout,self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis = 0)
        
        return dx
    
    

In [8]:
#softmax-with-loss
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None #손실
        self.y = None #softmax 의 출력
        self.t = None # 정답 레이블 (원핫벡터)
        
    def forward(self,x,t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y ,self.t)
        return self.loss
    
    def backward(self,dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y-self.t)/batch_size##역전파는 전파하는 값을 배치의 수로 나눠서 데이터 1개당 오차를 앞계층으로 전파한다!!! 
        return dx

In [9]:
#Two hidden layer (backpropagation) 
import sys,os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

In [10]:
class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

In [13]:
#기울기 검증하기 
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from dataset.mnist import load_mnist
#from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치의 절대 오차의 평균을 구한다.
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

b2:1.3970239658989137e-07
W1:2.3401499778134436e-06
W2:6.961034329968084e-09
b1:1.5605516269185457e-05


In [17]:
#two-hidden layer
import sys,os
sys.path.append(os.pardir)
from common.functions import *
from common.gradient import numerical_gradient

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size,weight_init_std=0.01):
        #가중치 초기화
        self.params = {}
        self.params["W1"] = weight_init_std * np.random.randn(input_size,hidden_size)#0과 1사이의 값을 랜덤하게 배열의 크기는 입력층 뉴런수 곱하기 은닉층 뉴런수
        self.params["b1"] = np.zeros(hidden_size)#은닉층의 사이즈 만큼 영인 배열 생성
        self.params["W2"] = weight_init_std * np.random.randn(hidden_size,output_size)
        self.params["b2"] = np.zeros(output_size)
    
    def predict(self,x): # 예측!!
        W1,W2 = self.params["W1"],self.params["W2"]
        b1,b2 = self.params['b1'],self.params["b2"]
        a1 = np.dot(x,W1)+b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1,W2)+b2
        y = softmax(a2)
        
        return y
    
    def loss(self,x,t): # 손실 함수 x는 입력, t는 정답 데이터
        y = self.predict(x)
        return cross_entropy_error(y,t)
    def accuracy(self,x,t): # 예측은 정확도를 계산
        y = self.predict(x)
        y = np.argmax(y,axis = 1)# argmax 는 배열중에서 가장 큰 값의 위치를 반환
        #axis 는  축 즉 배열의 차원 , rank는 축의 개수
        t = np.argmax(t,axis = 1)
        
        accuracy = np.sum(y==t)/float(x.shape[0])
        return accuracy
    def numerical_gradient(self,x,t): r
        loss_W = lambda W : self.loss(x,t)
        
        grads = {}#기울기를 딕셔너리로 저장
        grads['W1'] = numerical_gradient(loss_W, self.params["W1"])
        grads['b1'] = numerical_gradient(loss_W, self.params["b1"])
        grads['W2'] = numerical_gradient(loss_W, self.params["W2"])
        grads['b2'] = numerical_gradient(loss_W, self.params["b2"])
        
        return grads
    

IndentationError: unexpected indent (<ipython-input-17-aa41d25caa01>, line 38)

In [22]:
#오차역전파법을 사용한 학습 구현하기
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from dataset.mnist import load_mnist

#데이터 읽어오기 
(x_train,t_train),(x_test,t_test) = load_mnist(normalize = True, one_hot_label =True)
network = TwoLayerNet(input_size=784,hidden_size=50,output_size = 10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list=[]
train_acc_list = []
test_acc_list=[]

iter_per_epoch = max(train_size/batch_size,1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size,batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    #오차역전파법으로 기울기 구하기
    grad = network.gradient(x_batch,t_batch)
    
    #갱신
    for key in ("W1","b1","W2","b2"):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train,t_train)
        test_acc = network.accuracy(x_test,t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc,test_acc)

0.12316666666666666 0.1235
0.9042833333333333 0.9083
0.9241166666666667 0.926
0.9362666666666667 0.9352
0.9457 0.9427
0.9528666666666666 0.9478
0.9576333333333333 0.9534
0.9615333333333334 0.958
0.9646 0.9604
0.96725 0.962
0.96985 0.9633
0.97275 0.9674
0.9746666666666667 0.9678
0.9749833333333333 0.9687
0.9775666666666667 0.9693
0.9768666666666667 0.9674
0.9791333333333333 0.9693
